### imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from regex_to_NFA import NFA_CLASS,NFA
from NFA_to_DFA import DFA_CLASS,State,Edge
import json
from parser_classes import EPSILON

### Functions

In [4]:
def json_to_nfa(json_path: str) -> NFA:
    """
    This function is used to convert the JSON data to NFA
    params:
        json_path: str: Path to the JSON file
    return:
        NFA: NFA object
    """
    # read the JSON file
    with open(json_path, "r") as file:
        json_data = json.load(file)
    # create the NFA object
    nfa = NFA(None, None, [], [])
    # create the states
    states = []
    acceping_index = 0
    for state in json_data:
        if state == "startingState":
            continue
        if json_data[state]["isTerminatingState"] == True:
            acceping_index = int(state[1:])
        states.append(State(name=state))
    nfa.states = states
    # set the start state becuase the name of the starting states is the index of it inside the states array
    nfa.start = states[int(json_data["startingState"][1:])]
    # set the accept state
    nfa.accept = states[acceping_index]
    # create the transitions
    transitions = []
    for state in json_data:
        if state == "startingState":
            continue
        for transition in json_data[state]:
            if transition == "isTerminatingState":
                continue

            # check if the transition is epsilon or not
            if transition.startswith("epsilon"):
                transitions.append(
                    Edge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {EPSILON},
                    )
                )
            else:
                transitions.append(
                    Edge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {transition},
                    )
                )
    nfa.transitions = transitions
    return nfa

In [5]:
# getting the inputs from the tansitions in the nfa
def get_inputs(transitions):
    """_summary_
    This function is used to get the inputs from the transitions in the NFA
    Args:
        transitions (List): List of transitions in the NFA (List of Edges)
    Returns:
        List: List of unique inputs in the NFA
    """
    inputs = []
    for transition in transitions:
        temp_in = list(transition.characters)[0]
        if temp_in not in inputs and temp_in != EPSILON:
            inputs.append(temp_in)
    return inputs

### Regex to NFA

In [6]:
# regex = "[A-Za-z]+[0-9]*"
# regex = "ab*c+de?(f|g|h)|mr|n|[p-qs0-9]"
# regex = "ab"
regex = "(A|B)+"

In [7]:
nfa = NFA_CLASS(regex)

Visualization of the NFA is saved in nfa


### NFA to DFA

In [8]:
# deserialize the json file to get the nfa
deserialized_nfa = json_to_nfa("./nfa.json")

In [9]:
inputs = get_inputs(deserialized_nfa.transitions)

In [10]:
dfa = DFA_CLASS()

In [11]:
dfa.subset_construction(deserialized_nfa, inputs)

In [12]:
dfa.print_dfa()

Start State:  S2,S0,S4,S6
Accepting States:  ['S2,S5,S0,S1,S4,S6,S7', 'S2,S5,S0,S3,S4,S6,S7']
Non Accepting States:  ['S2,S0,S4,S6']
States:  ['S2,S0,S4,S6', 'S2,S5,S0,S1,S4,S6,S7', 'S2,S5,S0,S3,S4,S6,S7']
Transitions: 
From: S2,S0,S4,S6, To: S2,S5,S0,S1,S4,S6,S7, Characters: {'A'}
From: S2,S0,S4,S6, To: S2,S5,S0,S3,S4,S6,S7, Characters: {'B'}
From: S2,S5,S0,S1,S4,S6,S7, To: S2,S5,S0,S1,S4,S6,S7, Characters: {'A'}
From: S2,S5,S0,S1,S4,S6,S7, To: S2,S5,S0,S3,S4,S6,S7, Characters: {'B'}
From: S2,S5,S0,S3,S4,S6,S7, To: S2,S5,S0,S1,S4,S6,S7, Characters: {'A'}
From: S2,S5,S0,S3,S4,S6,S7, To: S2,S5,S0,S3,S4,S6,S7, Characters: {'B'}


In [13]:
dfa.rename_dfa_states()
dfa.print_dfa()

Start State:  S0
Accepting States:  ['S1', 'S2']
Non Accepting States:  ['S0']
States:  ['S0', 'S1', 'S2']
Transitions: 
From: S0, To: S1, Characters: {'A'}
From: S0, To: S2, Characters: {'B'}
From: S1, To: S1, Characters: {'A'}
From: S1, To: S2, Characters: {'B'}
From: S2, To: S1, Characters: {'A'}
From: S2, To: S2, Characters: {'B'}


In [14]:
dfa.dfa_to_json("dfa.json")

In [15]:
dfa.visualize_dfa()

Visualization of the DFA is saved in ./dfa


### Minimizing DFA

In [16]:
minimized_dfa = dfa.minimize_dfa(inputs)

In [17]:
# create a new object of DFA_CLASS to use the minimized dfa
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)

In [18]:
# print the minimized dfa
minimized_dfa_obj.print_dfa()

Start State:  S1
Accepting States:  ['S0']
Non Accepting States:  ['S1']
States:  ['S0', 'S1']
Transitions: 
From: S1, To: S0, Characters: {'A'}
From: S1, To: S0, Characters: {'B'}
From: S0, To: S0, Characters: {'A'}
From: S0, To: S0, Characters: {'B'}
From: S0, To: S0, Characters: {'A'}
From: S0, To: S0, Characters: {'B'}


In [19]:
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")

In [20]:
minimized_dfa_obj.visualize_dfa(path="minimized_dfa")

Visualization of the DFA is saved in minimized_dfa


### Testing

In [21]:
# input_regex = input("Enter the regular expression: ")
input_regex = "a*ba*ba*ba*"
nfa = NFA_CLASS(input_regex)
deserialized_nfa = json_to_nfa("./nfa.json")
inputs = get_inputs(deserialized_nfa.transitions)
dfa = DFA_CLASS()
dfa.subset_construction(deserialized_nfa, inputs)
dfa.rename_dfa_states()
dfa.dfa_to_json("dfa.json")
dfa.visualize_dfa()
# dfa.print_dfa()
minimized_dfa = dfa.minimize_dfa(inputs)
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)
minimized_dfa_obj.print_dfa()
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")
minimized_dfa_obj.visualize_dfa(path="minimized_dfa")

Visualization of the NFA is saved in nfa
Visualization of the DFA is saved in ./dfa
Start State:  S3
Accepting States:  ['S0']
Non Accepting States:  ['S1', 'S2', 'S3']
States:  ['S0', 'S1', 'S2', 'S3']
Transitions: 
From: S3, To: S3, Characters: {'a'}
From: S3, To: S1, Characters: {'b'}
From: S3, To: S3, Characters: {'a'}
From: S3, To: S1, Characters: {'b'}
From: S1, To: S1, Characters: {'a'}
From: S1, To: S2, Characters: {'b'}
From: S1, To: S1, Characters: {'a'}
From: S1, To: S2, Characters: {'b'}
From: S2, To: S2, Characters: {'a'}
From: S2, To: S0, Characters: {'b'}
From: S2, To: S2, Characters: {'a'}
From: S2, To: S0, Characters: {'b'}
From: S0, To: S0, Characters: {'a'}
From: S0, To: S0, Characters: {'a'}
Visualization of the DFA is saved in minimized_dfa
